# 0 설정, 설치 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

     |████████████████████████████████| 3.5 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
     |████████████████████████████████| 596 kB 29.6 MB/s 
     |████████████████████████████████| 6.8 MB 38.5 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install gluonnlp

     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595721 sha256=aa02b2195990ded1af4b198cc3fc3593f9e0b140bfe172fa3a04a8fa17560bbe
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [4]:
pip install mxnet

     |████████████████████████████████| 47.3 MB 83 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [5]:
pip install urllib3 --upgrade

     |████████████████████████████████| 138 kB 5.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-88kgebjo
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-88kgebjo
     |████████████████████████████████| 132 kB 5.5 MB/s 
     |████████████████████████████████| 4.5 MB 30.6 MB/s 
     |████████████████████████████████| 1.2 MB 35.8 MB/s 
     |████████████████████████████████| 127 kB 35.4 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 8.5 MB 33.3 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=9c55681bc3d625895e74f5052931fb81257a9d05672510ac6c97235402c6202e
  Stored in directory: /tmp/pip-ephem-wheel-cache-pd4phfib/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalle

In [7]:
import os
os.chdir('/content/drive/MyDrive/015GithubRepos/Dacon_sentence_classification')

import pandas as pd
import numpy as np
import re

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/.shortcut-targets-by-id/1dgpwd6oEOntxHkubWd_JYyE1sv712XJG/015GithubRepos/Dacon_sentence_classification/.cache/kobert_v1.zip
using cached model. /content/drive/.shortcut-targets-by-id/1dgpwd6oEOntxHkubWd_JYyE1sv712XJG/015GithubRepos/Dacon_sentence_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 경로지정

In [9]:
dataPATH = 'data/'

## device 설정

In [10]:
#GPU 사용
device = torch.device("cuda:0")

# 1 하이퍼 파라미터 설정

In [11]:
max_len = 205
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

learning_rate

5e-05

# 2 Data 처리

## 텍스트 전처리

In [12]:
# dev 데이터 더함

def load_data(path):
    train = pd.read_csv(path+'train_data.csv')
    test = pd.read_csv(path+'test_data.csv')
    train_dev = pd.read_csv(path+'train_dev.csv')
    new_df = pd.read_csv(path+'new_df.csv')
    sample_submission = pd.read_csv(path+'sample_submission.csv')

    train = pd.concat([train,train_dev], ignore_index=True)
    train = pd.concat([train,new_df], ignore_index=True)

    label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}

    train['label'] = train['label'].map(label_dict)
    train['premise']=train['premise'].map(lambda x: re.sub('[-=+.,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', x))
    train['hypothesis']=train['hypothesis'].map(lambda x: re.sub('[-=+.,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', x))
    test['premise']=test['premise'].map(lambda x: re.sub('[-=+.,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', x))
    test['hypothesis']=test['hypothesis'].map(lambda x: re.sub('[-=+.,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', x))

    return train,test,sample_submission 

def text_clean(df):
    df["premise_"] = "[CLS]" + df["premise"] + "[SEP]"
    df["hypothesis_"] = df["hypothesis"] + "[SEP]"
    df["text_sum"] = df.premise_ + " " + df.hypothesis_
    df = df[['text_sum','label']]

    return df 

train, test, sample_submission = load_data(dataPATH)
clean_train, clean_test = text_clean(train), text_clean(test)


In [13]:
clean_train

,text_sum,label
0,[CLS]씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서 소년이나 ...,1
1,[CLS]삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 ...,1
2,[CLS]이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다[SEP] 예측적 범...,0
3,[CLS]광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민...,2
4,[CLS]진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면 이런 상...,2
...,...,...
30871,[CLS]GAO는 매일 웹사이트에 오늘의 보고서로 알려진 이 목록을 게시한다[SEP...,0
30872,[CLS]GAO는 매일 웹사이트에 오늘의 보고서로 알려진 이 목록을 게시한다[SEP...,1
30873,[CLS]너무 작아서 상관없어[SEP] 형사가 작다고 무시하는 것은 좋지 않다[SEP],0
30874,[CLS]너무 작아서 상관없어[SEP] 그것은 형사가 작은 것들을 무시하면서 할 수...,2


## CustomDataset 클래스 선언

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair, train):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer,
            max_seq_length=max_len,
            pad=pad,
            pair=pair
            )
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.train = train
        if self.train == 'train':
            self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        if self.train == 'train':
            return (self.sentences[i] + (self.labels[i], ))
        else:
            return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [31]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(data_list_train, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)

using cached model. /content/drive/.shortcut-targets-by-id/1dgpwd6oEOntxHkubWd_JYyE1sv712XJG/015GithubRepos/Dacon_sentence_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## DataLoader

In [29]:
data_list_train = []
for q, label in zip(clean_train['text_sum'], clean_train['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list_train.append(data)

data_list_test = []
for q, label in zip(clean_test['text_sum'], clean_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list_test.append(data)

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(data_list_train, 0, 1, tok, max_len, True, False, 'train')
data_test = BERTDataset(data_list_test, 0, 1, tok, max_len, True, False, 'test')

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/drive/.shortcut-targets-by-id/1dgpwd6oEOntxHkubWd_JYyE1sv712XJG/015GithubRepos/Dacon_sentence_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
data_train.sentences

# 3 모델 선언

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [33]:
# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# 4 학습

## 학습

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # model.eval()
    # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    #     token_ids = token_ids.long().to(device)
    #     segment_ids = segment_ids.long().to(device)
    #     valid_length= valid_length
    #     label = label.long().to(device)
    #     out = model(token_ids, valid_length, segment_ids)
    #     test_acc += calc_accuracy(out, label)
    # print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/965 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.063614010810852 train acc 0.5
epoch 1 batch id 201 loss 0.6834591031074524 train acc 0.6899875621890548
epoch 1 batch id 401 loss 0.4947221577167511 train acc 0.7194513715710723


In [ ]:
torch.save(model, 'kobert205_30876.pth')

## test 예측

In [ ]:
import time

# test_dataset = CustomDataset(clean_test,'test', modelname)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)


preds = []

start = time.time()

model.eval()
answer = []
with torch.no_grad():
    for token_ids, valid_length, segment_ids in tqdm(test_dataloader):
        y_pred = model(token_ids.to(device), valid_length.to(device), segment_ids.to(device)).detach().cpu().numpy()
        answer.extend(y_pred.argmax(axis=1))
        
preds.append(answer)
print(f'elapsed time : {time.time() - start}')

  0%|          | 0/27 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 27/27 [00:15<00:00,  1.75it/s]

elapsed time : 15.464909791946411
